In [ ]:
import aoc
import re

In [ ]:
example_target_string = 'target area: x=20..30, y=-10..-5'

In [ ]:
class ProbeTarget:
    def __init__(self, target_area_string):
        pattern = r'target area: x=(-*)(\d*)..(-*)(\d*), y=(-*)(\d*)..(-*)(\d*)'
        parsed_string = re.findall(pattern, target_area_string)[0]
        self.min_x, self.max_x = int(parsed_string[1]), int(parsed_string[3])
        self.min_y, self.max_y = int(parsed_string[5]), int(parsed_string[7])
        if parsed_string[0] == '-':
            self.min_x *= -1
        if parsed_string[2] == '-':
            self.max_x *= -1
        if parsed_string[4] == '-':
            self.min_y *= -1
        if parsed_string[6] == '-':
            self.max_y *= -1
        
        self.repr = f'target area: x={self.min_x}..{self.max_x}'
        self.repr += f', y={self.min_y}..{self.max_y}'
    
    def contains_point(self, point):
        if point[0] < self.min_x or point[0] > self.max_x:
            return False
        if point[1] < self.min_y or point[1] > self.max_y:
            return False
        return True
        
    def behind_point(self, point):
        return point[0] > self.max_x
    
    def above_point(self, point):
        return point[1] < self.min_y
    
    def __repr__(self):
        return self.repr

In [ ]:
example_target = ProbeTarget(example_target_string)

In [ ]:
example_target 

In [ ]:
class Probe:
    def __init__(self, initial_point, initial_velocities):
        self.initial_point = initial_point
        self.initial_velocities = initial_velocities
        self.reset()
        
    def reset(self):
        self.x = self.initial_point[0]
        self.y = self.initial_point[1]
        self.x_velocity = self.initial_velocities[0]
        self.y_velocity = self.initial_velocities[1]
        
    def step(self):
        self.x += self.x_velocity
        self.y += self.y_velocity
        
        if self.x_velocity < 0:
            self.x_velocity += 1
        elif self.x_velocity > 0:
            self.x_velocity -= 1
            
        self.y_velocity -= 1
        
    def point(self):
        return [self.x, self.y]
        
    def cannot_reach_target(self, target):
        if target.behind_point(self.point()):
            return True
        if target.above_point(self.point()) and self.y_velocity <= 0:
            return True
        return False
    
    def in_target(self, target):
        return target.contains_point(self.point())
        
    def path_towards_target(self, target):
        path = [self.point()]
        while not self.cannot_reach_target(target) and not self.in_target(target):
            self.step()
            path.append(self.point())
        return path
    
    def reaches_target(self, target):
        self.reset()
        path = self.path_towards_target(target)
        return target.contains_point(path[-1])
    
    def trickshot_score(self, target):
        self.reset()
        path = self.path_towards_target(target)
        if target.contains_point(path[-1]):
            return max([p[1] for p in path])
        else:
            return -999

In [ ]:
test_probe = Probe((0,0),(7,2))

In [ ]:
test_probe.path_towards_target(example_target)

In [ ]:
Probe((0,0),(6,3)).path_towards_target(example_target)

In [ ]:
Probe((0,0),(9,0)).reaches_target(example_target)

In [ ]:
Probe((0,0),(17,-4)).reaches_target(example_target)

In [ ]:
Probe((0,0),(17,-4)).trickshot_score(example_target)

In [ ]:
Probe((0,0),(6,9)).trickshot_score(example_target)

In [ ]:
def trickshot(target):
    best_velocities = (0,0)
    best_height = Probe((0,0),(0,0)).trickshot_score(target)
    for x in range(target.max_x):
        for y in range(-1000, 1000):
            height = Probe((0,0),(x,y)).trickshot_score(target)
            if height > best_height:
                best_height = height
                best_velocities = (x, y)
                print(f'New best {best_velocities} attains height {best_height}')
    return best_height

In [ ]:
%time trickshot(example_target)

In [ ]:
day17_target = ProbeTarget(aoc.read_file_as_string('inputs/day17.txt'))

In [ ]:
day17_target

In [ ]:
%time trickshot(day17_target)

In [ ]:
def find_successes(target):
    num_success = 0
    successes = []
    for x in range(target.max_x + 1):
        for y in range(-500, 500):
            if Probe((0,0),(x,y)).reaches_target(target):
                num_success += 1
                successes.append((x,y))
    return successes

In [ ]:
%time hits_found = find_successes(example_target)

In [ ]:
expected_string = r'''23,-10  25,-9   27,-5   29,-6   22,-6   21,-7   9,0     27,-7   24,-5
25,-7   26,-6   25,-5   6,8     11,-2   20,-5   29,-10  6,3     28,-7
8,0     30,-6   29,-8   20,-10  6,7     6,4     6,1     14,-4   21,-6
26,-10  7,-1    7,7     8,-1    21,-9   6,2     20,-7   30,-10  14,-3
20,-8   13,-2   7,3     28,-8   29,-9   15,-3   22,-5   26,-8   25,-8
25,-6   15,-4   9,-2    15,-2   12,-2   28,-9   12,-3   24,-6   23,-7
25,-10  7,8     11,-3   26,-7   7,1     23,-9   6,0     22,-10  27,-6
8,1     22,-8   13,-4   7,6     28,-6   11,-4   12,-4   26,-9   7,4
24,-10  23,-8   30,-8   7,0     9,-1    10,-1   26,-5   22,-9   6,5
7,5     23,-6   28,-10  10,-2   11,-1   20,-9   14,-2   29,-7   13,-3
23,-5   24,-8   27,-9   30,-7   28,-5   21,-10  7,9     6,6     21,-5
27,-10  7,2     30,-9   21,-8   22,-7   24,-9   20,-6   6,9     29,-5
8,-2    27,-8   30,-5   24,-7'''

In [ ]:
expected = [(int(p[0]), int(p[1])) 
            for p in re.findall(r'(\d+),(\d+)', expected_string)]

expected.extend([(int(p[0]), -1*int(p[1])) 
            for p in re.findall(r'(\d+),-(\d+)', expected_string)])

In [ ]:
len(expected)

In [ ]:
len(hits_found)

In [ ]:
[p for p in expected if p not in hits_found]

In [ ]:
%time hits_found = find_successes(day17_target)

In [ ]:
len(hits_found)